In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import re
from pprint import pprint
import db_mysql as db_mysql

In [2]:
import MySQLdb

hostname = "localhost"
username = "root"
password = ""
db_name  = "drama_ojol"

# Open database connection
db = MySQLdb.connect(hostname,username,password,db_name)
db.set_character_set('utf8')

# prepare a cursor object using cursor() method
cursor = db.cursor()
cursor.execute('SET NAMES utf8;')
cursor.execute('SET CHARACTER SET utf8;')
cursor.execute('SET character_set_connection=utf8;')

def executeSql(sql):
    return cursor.execute(sql);

def fetch(type='all'):
    if type=='all':
        return cursor.fetchall();
    else:
        return cursor.fetchone();
def commit():
    db.commit();

def escapeString(string):
    return MySQLdb.escape_string(string);

# Preprocessing Mentions

In [3]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = re.sub('\.+','.',tweet);
    tweet = re.sub('\,+',',',tweet);
    tweet = re.sub('\"+','"',tweet); #Remove tanda ""
    tweet = tweet.replace("/ ","/");
    tweet = tweet.replace("www. ","www.");
    tweet = re.sub(r'RT', '', tweet);
    tweet = re.sub(r'http\S+', '', tweet, flags=re.MULTILINE);
    tweet = re.sub(r'@([A-Za-z0-9_]+)', '', tweet, flags=re.MULTILINE);
    tweet = re.sub(r'pic.twitter.com/\S+', '', tweet, flags=re.MULTILINE);
    tweet = tweet.replace("?"," ? ");
    tweet = tweet.replace("-"," ");
    tweet = tweet.replace("/"," ");
    tweet = tweet.replace("["," ");
    tweet = tweet.replace("]"," ");
    tweet = tweet.replace("."," ");
    tweet = tweet.replace(","," ");
    tweet = re.sub(r'[^a-z0-9.,?# -]', ' ', tweet, flags = re.IGNORECASE|re.MULTILINE)
    tweet = re.sub(" \d+", " ", tweet)
    tweet = re.sub(r'( +)', ' ', tweet, flags = re.IGNORECASE|re.MULTILINE)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

# Read the tweets one by one and process it

In [4]:
dokumen = []
docs, index = [], []
fp = open('data_tes.csv', 'r')
line = fp.readline()
while line:
    processedTweet = processTweet(line)
    dokumen.append(processedTweet.split(' '))
    line = fp.readline()
fp.close()

In [5]:
datatest = pd.read_csv("data/kategori/datatest.csv",sep=",", header=None)
datatest.columns = ['kategori']

# print(datatest['kategori'][1])
# if(datasets['kategori'][0]=="aplikas"):
#     print("ada")

# Make dataframe containing list of word each category

In [6]:
df_index = pd.DataFrame({
    'kata':datatest['kategori']
})

# Count word accurance in each document

In [7]:
for i in range(len(dokumen)):
    terdeteksi = []
    for kata in datatest['kategori']:
        if kata in dokumen[i]:
            nilai=1
        else:
            nilai=0
        terdeteksi.append(nilai)
        df_index[i+1] = pd.Series(terdeteksi)

In [8]:
df_index.to_csv("data/matriks_datatest.csv", index=False)
# df_index = pd.read_csv("data/matriks_datatest.csv",sep=",")
# df_index = df_tmp.drop(['kata'], axis=1)
df_index

,kata,1,2,3,4,5,6,7,8,9,...,832,833,834,835,836,837,838,839,840,841
0,tarif,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tarifnya,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,murah,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,mahal,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,harga,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,argo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,voucher,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,diskon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ongkos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ongkosnya,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_index_baru = df_index.drop(['kata'], axis=1)
df_index_baru

,1,2,3,4,5,6,7,8,9,10,...,832,833,834,835,836,837,838,839,840,841
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Open the aggregration matrix

In [10]:
category = pd.read_csv("data/aggregasi_matriks.csv",sep=",", header=0)
list_cat = category.drop(['kategori'], axis=1)
list_cat

,tarif,tarifnya,murah,mahal,harga,argo,voucher,diskon,ongkos,ongkosnya,...,kompetitor,respon,memecat,recruitment,mengatasnamakan,admin,layanan,penipuan,marketing,sop
0,0.092105,0.017802,0.04257,0.033282,0.06192,0.008514,0.099071,0.0387,0.017802,0.004644,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.002052,0.035916,0.000513,0.001026,0.007696,0.030785,0.042073,0.018984,0.003079,0.00667


In [11]:
list_category = list_cat.values.tolist()
for category in list_category:
    for item in category:
        float(item)

# Assign matrix of each category into variables

In [12]:
cat_harga = []
cat_aplikasi = []
cat_driver = []
cat_penumpang = []
cat_perusahaan = []
cat_harga = np.array(list_category[0])
cat_aplikasi = np.array(list_category[1])
cat_driver = np.array(list_category[2])
cat_penumpang = np.array(list_category[3])
cat_perusahaan = np.array(list_category[4])

# Convert datatest matrix into numpy array

In [13]:
# temp = df_index.drop(['kata'], axis=1)
temp = df_index_baru
temp_datatest = temp.values.tolist()
datatest = np.array(temp_datatest)
# datatest[:,0].shape

In [14]:
temp

,1,2,3,4,5,6,7,8,9,10,...,832,833,834,835,836,837,838,839,840,841
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Fungsi Cosine Similarity

In [15]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

# Categorisasi dengan cosine

In [18]:
import math
total_label = []
total_datatest = np.size(datatest,1)
for i in range(0,total_datatest):
    harga = cos_sim(datatest[:,i], cat_harga)
    aplikasi = cos_sim(datatest[:,i], cat_aplikasi)
    driver = cos_sim(datatest[:,i], cat_driver)
    penumpang = cos_sim(datatest[:,i], cat_penumpang)
    perusahaan = cos_sim(datatest[:,i], cat_perusahaan)
    maximal = max(harga,aplikasi,driver,penumpang,perusahaan)
    if maximal == harga and math.isnan(maximal)==False:
        label = "harga"
    if maximal == aplikasi and math.isnan(maximal)==False:
        label = "aplikasi"
    elif maximal == driver and math.isnan(maximal)==False:
        label = "driver"
    elif maximal == penumpang and math.isnan(maximal)==False:
        label = "penumpang"
    elif maximal == perusahaan and math.isnan(maximal)==False:
        label = "perusahaan"
    elif math.isnan(maximal)==True:
        label = "others"
    print(5302+i+1,"=",label,maximal)
    print(i+1,"=",label,maximal)   
   
    
    sql = '''UPDATE opinions
    		 SET cat_result="%s" WHERE id=%d''' % \
    		 (label,5302+i+1);
    executeSql(sql);
    commit();

5303 = penumpang 0.504106769658824
1 = penumpang 0.504106769658824
5304 = aplikasi 0.49305777195152106
2 = aplikasi 0.49305777195152106
5305 = aplikasi 0.34254834723571626
3 = aplikasi 0.34254834723571626
5306 = others nan
4 = others nan
5307 = penumpang 0.44905133855934154
5 = penumpang 0.44905133855934154
5308 = aplikasi 0.25892296558733047
6 = aplikasi 0.25892296558733047


C:\Users\Brandon\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


5309 = others nan
7 = others nan
5310 = driver 0.9902528390930346
8 = driver 0.9902528390930346
5311 = perusahaan 0.3197041021669204
9 = perusahaan 0.3197041021669204
5312 = driver 0.5799380245462367
10 = driver 0.5799380245462367
5313 = aplikasi 0.5512553474590951
11 = aplikasi 0.5512553474590951
5314 = perusahaan 0.6366744280911469
12 = perusahaan 0.6366744280911469
5315 = driver 0.040579812711670854
13 = driver 0.040579812711670854
5316 = perusahaan 0.4501968055113169
14 = perusahaan 0.4501968055113169
5317 = aplikasi 0.4844365184292383
15 = aplikasi 0.4844365184292383
5318 = others nan
16 = others nan
5319 = driver 0.5717227432161548
17 = driver 0.5717227432161548
5320 = driver 0.5717227432161548
18 = driver 0.5717227432161548
5321 = aplikasi 0.4844365184292383
19 = aplikasi 0.4844365184292383
5322 = others nan
20 = others nan
5323 = aplikasi 0.06681882902985685
21 = aplikasi 0.06681882902985685
5324 = driver 0.5717227432161548
22 = driver 0.5717227432161548
5325 = driver 0.7002144

5445 = penumpang 0.35429275320575393
143 = penumpang 0.35429275320575393
5446 = penumpang 0.0940303031923974
144 = penumpang 0.0940303031923974
5447 = driver 0.9902528390930346
145 = driver 0.9902528390930346
5448 = driver 0.7661740325029098
146 = driver 0.7661740325029098
5449 = harga 0.12234017852796977
147 = harga 0.12234017852796977
5450 = aplikasi 0.13005807875467051
148 = aplikasi 0.13005807875467051
5451 = others nan
149 = others nan
5452 = aplikasi 0.49531228895889556
150 = aplikasi 0.49531228895889556
5453 = driver 0.5717227432161548
151 = driver 0.5717227432161548
5454 = driver 0.9902528390930346
152 = driver 0.9902528390930346
5455 = aplikasi 0.45009810641847686
153 = aplikasi 0.45009810641847686
5456 = driver 0.5799380245462367
154 = driver 0.5799380245462367
5457 = driver 0.4951264195465173
155 = driver 0.4951264195465173
5458 = penumpang 0.436569268744252
156 = penumpang 0.436569268744252
5459 = aplikasi 0.34254834723571626
157 = aplikasi 0.34254834723571626
5460 = harga 

5578 = penumpang 0.6174021807734257
276 = penumpang 0.6174021807734257
5579 = aplikasi 0.5703514343250139
277 = aplikasi 0.5703514343250139
5580 = others nan
278 = others nan
5581 = harga 0.43686322598956484
279 = harga 0.43686322598956484
5582 = harga 0.7201755146245009
280 = harga 0.7201755146245009
5583 = driver 0.9902528390930346
281 = driver 0.9902528390930346
5584 = harga 0.7201755146245009
282 = harga 0.7201755146245009
5585 = perusahaan 0.531728509336657
283 = perusahaan 0.531728509336657
5586 = perusahaan 0.531728509336657
284 = perusahaan 0.531728509336657
5587 = perusahaan 0.06015821370867749
285 = perusahaan 0.06015821370867749
5588 = others nan
286 = others nan
5589 = others nan
287 = others nan
5590 = driver 0.5717227432161548
288 = driver 0.5717227432161548
5591 = driver 0.9902528390930346
289 = driver 0.9902528390930346
5592 = others nan
290 = others nan
5593 = driver 0.9902528390930346
291 = driver 0.9902528390930346
5594 = harga 0.5793639541469566
292 = harga 0.579363

5710 = others nan
408 = others nan
5711 = others nan
409 = others nan
5712 = perusahaan 0.4501968055113169
410 = perusahaan 0.4501968055113169
5713 = perusahaan 0.3890024411641926
411 = perusahaan 0.3890024411641926
5714 = perusahaan 0.6366744280911469
412 = perusahaan 0.6366744280911469
5715 = driver 0.5717227432161548
413 = driver 0.5717227432161548
5716 = others nan
414 = others nan
5717 = driver 0.07974768617173088
415 = driver 0.07974768617173088
5718 = driver 0.7002144976119159
416 = driver 0.7002144976119159
5719 = driver 0.09328086881066494
417 = driver 0.09328086881066494
5720 = others nan
418 = others nan
5721 = others nan
419 = others nan
5722 = driver 0.5717227432161548
420 = driver 0.5717227432161548
5723 = driver 0.5717227432161548
421 = driver 0.5717227432161548
5724 = perusahaan 0.07018458200874396
422 = perusahaan 0.07018458200874396
5725 = driver 0.6255784779342344
423 = driver 0.6255784779342344
5726 = driver 0.9902528390930346
424 = driver 0.9902528390930346
5727 = 

5850 = driver 0.4428545326248373
548 = driver 0.4428545326248373
5851 = others nan
549 = others nan
5852 = penumpang 0.0756160156186688
550 = penumpang 0.0756160156186688
5853 = driver 0.7002144976119159
551 = driver 0.7002144976119159
5854 = driver 0.9902528390930346
552 = driver 0.9902528390930346
5855 = penumpang 0.2585833362305972
553 = penumpang 0.2585833362305972
5856 = aplikasi 0.25227082599881606
554 = aplikasi 0.25227082599881606
5857 = perusahaan 0.6943260487477381
555 = perusahaan 0.6943260487477381
5858 = driver 0.9902528390930346
556 = driver 0.9902528390930346
5859 = perusahaan 0.4501968055113169
557 = perusahaan 0.4501968055113169
5860 = others nan
558 = others nan
5861 = aplikasi 0.10630810664101646
559 = aplikasi 0.10630810664101646
5862 = perusahaan 0.6366744280911469
560 = perusahaan 0.6366744280911469
5863 = others nan
561 = others nan
5864 = driver 0.040579812711670854
562 = driver 0.040579812711670854
5865 = others nan
563 = others nan
5866 = perusahaan 0.36758415

5991 = driver 0.06595953489099389
689 = driver 0.06595953489099389
5992 = perusahaan 0.7519776694043293
690 = perusahaan 0.7519776694043293
5993 = driver 0.7002144976119159
691 = driver 0.7002144976119159
5994 = penumpang 0.6174021807734257
692 = penumpang 0.6174021807734257
5995 = driver 0.9902528390930346
693 = driver 0.9902528390930346
5996 = others nan
694 = others nan
5997 = others nan
695 = others nan
5998 = aplikasi 0.4331292020805764
696 = aplikasi 0.4331292020805764
5999 = penumpang 0.504106769658824
697 = penumpang 0.504106769658824
6000 = driver 0.5717227432161548
698 = driver 0.5717227432161548
6001 = driver 0.005270105879096401
699 = driver 0.005270105879096401
6002 = aplikasi 0.18308618477173244
700 = aplikasi 0.18308618477173244
6003 = aplikasi 0.10630810664101646
701 = aplikasi 0.10630810664101646
6004 = perusahaan 0.08221622553213519
702 = perusahaan 0.08221622553213519
6005 = driver 0.02582351705779191
703 = driver 0.02582351705779191
6006 = driver 0.9902528390930346


6124 = aplikasi 0.31048717142256554
822 = aplikasi 0.31048717142256554
6125 = others nan
823 = others nan
6126 = others nan
824 = others nan
6127 = harga 0.17301513969740057
825 = harga 0.17301513969740057
6128 = driver 0.5717227432161548
826 = driver 0.5717227432161548
6129 = harga 0.4526421249834078
827 = harga 0.4526421249834078
6130 = harga 0.7201755146245009
828 = harga 0.7201755146245009
6131 = aplikasi 0.17062665718820277
829 = aplikasi 0.17062665718820277
6132 = driver 0.4428545326248373
830 = driver 0.4428545326248373
6133 = driver 0.4428545326248373
831 = driver 0.4428545326248373
6134 = aplikasi 0.5768349038081602
832 = aplikasi 0.5768349038081602
6135 = perusahaan 0.434154509855177
833 = perusahaan 0.434154509855177
6136 = driver 0.47514355396353275
834 = driver 0.47514355396353275
6137 = perusahaan 0.531728509336657
835 = perusahaan 0.531728509336657
6138 = perusahaan 0.531728509336657
836 = perusahaan 0.531728509336657
6139 = driver 0.5717227432161548
837 = driver 0.57172